In [1]:
from flask import Flask, render_template, request
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import os
import io
import base64
from tensorflow.keras.models import load_model
from pyngrok import ngrok, conf
import getpass
import threading
from flasgger import Swagger
from flask_cors import CORS
from datetime import datetime



2025-04-09 08:51:21.695517: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
code_path = '/Volumes/Datasets/HandGesture/models'
model = load_model(os.path.join(code_path,'HagridModel1.keras'))

In [3]:
os.environ["FLASK_DEBUG"] = "development"

app = Flask(__name__)
swagger = Swagger(app)
CORS(app)
port = 5001


# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port, bind_tls=True).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")


# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# Preprocessing function for live image input (adjust as per your model)
def preprocess_image(img):
    # Decode the image into a tensor
    img = tf.convert_to_tensor(img, dtype=tf.float32)
    img_resized = tf.image.resize(img, [224, 224])
    # Preprocess the image using MobileNet's preprocessing function
    img_preprocessed = tf.keras.applications.mobilenet.preprocess_input(img_resized)
    # Add a batch dimension
    img_batch = tf.expand_dims(img_preprocessed, axis=0)

    return img_batch, img_resized



def predict_model(img_batch, img_resized):
    # Perform the prediction
    prediction = model.predict(img_batch)

    buffer = io.BytesIO()
    plt.imshow(tf.cast(img_resized, tf.uint8))

    labels = ['Dislike', 'Like', 'Mute', 'OK', 'Stop']
    plt.title(labels[prediction.argmax()])
    plt.axis('off')
    plt.savefig(buffer, format='png')
    plt.close()
    #Encode the image to base64 string
    buffer.seek(0)
    image_string = base64.b64encode(buffer.read()).decode('utf-8')

    return labels[prediction.argmax()], image_string


# Define the prediction route
@app.route('/', methods=['GET', 'POST'])
def predict():

    chart_url = None
    if request.method == 'POST':

        # Get the image from the request
        file = request.files['file']
        # Read the image for processing without saving it first
        img = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_COLOR)

        # Preprocess the image
        img_batch, img_resized = preprocess_image(img)
        
        # Get the prediction name and the prediction image
        prediction_name, img_prediction = predict_model(img_batch, img_resized)

        # Construct the filename using the prediction name
        filename = f"Upload_{prediction_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
        path = '/Volumes/Datasets/SavedImages/'
        save_path = os.path.join(path, filename)

        # Save the image with the prediction name
        cv2.imwrite(save_path, img)

        # Prepare the data for displaying in HTML (data:image/png;base64, ...)
        chart_url = f"{img_prediction}"

    return render_template('handgestureIndex.html', chart_url =chart_url) #,int(predicted_class[0])


# Define the capture route for webcam images
@app.route('/capture', methods=['POST'])
def capture():
    data = request.form['image_base64']

    # Decode the base64 image
    image_data = data.split(',')[1]  # Strip the data:image/jpeg;base64, header
    decoded_image = base64.b64decode(image_data)

    # Convert to NumPy array and decode
    npimg = np.frombuffer(decoded_image, np.uint8)
    img = cv2.imdecode(npimg, cv2.IMREAD_COLOR)
    

    # Preprocess the image
    img_batch, img_resized = preprocess_image(img)
    prediction_name, img_prediction = predict_model(img_batch, img_resized)
    
    filename = f"webcam_{prediction_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
    path = '/Volumes/Datasets/SavedImages'
    save_path = os.path.join(path, filename)
    cv2.imwrite(save_path, img)

    # Convert prediction image to base64 for display
    chart_url = f"{img_prediction}"

    return {'chart_url': chart_url}  # Send back as a JSON response


# Run the Flask app
if __name__ == '__main__':
    threading.Thread(target=app.run, kwargs={"host": "0.0.0.0", "port": 5001, "use_reloader": False}).start()


 * ngrok tunnel "https://24ad-2600-4041-7ad1-3200-255b-10cf-72e8-d1b0.ngrok-free.app" -> "http://127.0.0.1:5001"
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.1.176:5001
Press CTRL+C to quit
127.0.0.1 - - [09/Apr/2025 08:51:26] "GET /socket.io/?EIO=4&transport=polling&t=POQclbL HTTP/1.1" 404 -
127.0.0.1 - - [09/Apr/2025 08:51:32] "GET /socket.io/?EIO=4&transport=polling&t=POQcn35 HTTP/1.1" 404 -
127.0.0.1 - - [09/Apr/2025 08:51:38] "GET /socket.io/?EIO=4&transport=polling&t=POQcoWr HTTP/1.1" 404 -
127.0.0.1 - - [09/Apr/2025 08:51:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2025 08:51:41] "GET /static/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2025 08:51:41] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Apr/2025 08:51:44] "GET /socket.io/?EIO=4&transport=polling&t=POQcp-c HTTP/1.1" 404 -
127.0.0.1 - - [09/Apr/2025 08:51:50] "GET /socket.io/?EIO=4&transport=polling&t=POQcrSL HTTP/1.1" 404 -
127.0.0.1 - - [09/Apr/2025 08:51:56] "GET /socket.io/?EIO=4&transport=polling&t=POQcsw5 HTTP/1.1" 404 -
127.0.0.1 - - [09/Apr/2025 08:52: